# Ventilando datos abiertos sobre calidad del aire

This notebook follows the code in this [url](https://datos.gob.mx/blog/ventilando-datos-abiertos-sobre-calidad-del-aire)

It is convenient to have the `datosgobmx` client installed to download data. The client code can be seen in [this Gitub url](https://github.com/mxabierto/api_python_client)

```bash
pip install datosgobmx
```

Structure
* Descargar los datos de parametros de calidad del aire que se miden. 
* Análisis exploratorio de los datos con el objetivo de entender mejor su naturaleza y mostrar los hallazgos más interesantes. 
* En la tercera parte, un modelo predictivo sencillo para el ozono, usando los registros históricos de este contaminante en las ciudades de México y Guadalajara. 
* Finalmente, algunas ideas sobre análisis más profundos que pueden ser de interés tanto para el público en general como para especialistas.

<https://datos.gob.mx/blog/ventilando-datos-abiertos-sobre-calidad-del-aire>

y 

<https://github.com/mxabierto/calidad-del-aire>

## Parametros

In [5]:
# Vamos a utilizar estas librerías
import requests
import pandas as pd
import requests
import json
import numpy as np
from datosgobmx import client   # https://github.com/mxabierto/api_python_client
import mplleaflet
import folium
from glob import glob
from joblib import Parallel, delayed

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Obtener los parámetros de calidad del aire que se miden en las estaciones
# 1) Get the records through API request
# 2) Each record goes into its own DataFrame
# 3) Concatenate/bundll all the records/DataFrame into a single DataFrame

parametros_request = client.makeCall('sinaica-parametros')
# the above is the same than: https://api.datos.gob.mx/v2/sinaica-parametros

valid_params = []   # starts as a list, will end up as a DataFrame

for v in parametros_request['results']:
    # parametros_request['results'] is a list of dictionaries
    # https://stackoverflow.com/questions/26074447/creating-a-pandas-dataframe-from-a-dictionary
    aux = pd.DataFrame([v])     # each record goes into its own DataFrame
    valid_params.append(aux)    # put all the DataFrame into a list 

# Concatenate/bundll all the records/DataFrame into a single DataFrame
# this is the end result, here will have all the records together in a single DataFrame
valid_params = pd.concat(valid_params, ignore_index=True)

# convert 'date-insert' to a date
valid_params['date-insert'] = pd.to_datetime(valid_params['date-insert'])
#valid_params.info() # if you want to verify that date-insert is now datetime 
print('Shape (rows, columns): ' + str(valid_params.shape))
valid_params        # this is a DataFrame, contains the air attributes measured (e.g. CO, Ozone, etc.)


Shape (rows, columns): (7, 3)


,_id,parametro,date-insert
0,5c2aa3dfe2705c1932134299,CO,2018-12-31 23:18:55.923000+00:00
1,5c2aa3dfe2705c193213429a,O3,2018-12-31 23:18:55.923000+00:00
2,5c2aa3dfe2705c193213429b,PM10,2018-12-31 23:18:55.923000+00:00
3,5c2aa3dfe2705c193213429c,SO2,2018-12-31 23:18:55.923000+00:00
4,5c2aa3dfe2705c193213429d,NO2,2018-12-31 23:18:55.923000+00:00
5,5c2aa3dfe2705c193213429e,PM2.5,2018-12-31 23:18:55.923000+00:00
6,5c2aa3dfe2705c193213429f,TMP,2018-12-31 23:18:55.923000+00:00


## Estaciones

In [19]:
# Retrieve measurement stations. There are 185 in total

estaciones_request = client.makeCall('sinaica-estaciones', {'pageSize':200})
# the above is the same than: https://api.datos.gob.mx/v2/sinaica-estaciones?pageSize=200

estaciones = []

for v in estaciones_request['results']:
    aux = pd.DataFrame([v])
    estaciones.append(aux)      # estaciones is a list of DataFrames

# concatenate all the individual DataFrames into a single DataFrame
estaciones = pd.concat(estaciones, ignore_index=True)   # DataFrame with all the estaciones

#estaciones.info()
print('Shape (rows, columns): ' + str(estaciones.shape))
estaciones.head()       # show the first 5 entries

Shape (rows, columns): (185, 7)


,_id,lat,long,id,nombre,codigo,redesid
0,5cccf5bee2705c1932820580,21.873311,-102.320803,31,CBTIS,CBT,30
1,5cccf5bee2705c1932820581,21.846392,-102.288431,32,Secretaría de Medio Ambiente,SMA,30
2,5cccf5bee2705c1932820582,21.883781,-102.295825,33,Centro,CEN,30
3,5cccf5bee2705c1932820583,32.639722,-115.506389,39,COBACH,SPABC14,32
4,5cccf5bee2705c1932820584,32.603639,-115.485944,41,CESPM,SPABC19,32


In [7]:
estaciones.describe()     # descriptive statistics

,lat,long,id,redesid
count,185.000000,185.000000,185.000000,185.000000
mean,21.841909,-99.588421,200.989189,78.551351
std,5.331746,15.634371,118.276986,32.903661
min,0.000000,-117.056111,31.000000,30.000000
25%,19.371667,-103.312592,103.000000,49.000000
50%,20.558056,-100.344444,170.000000,72.000000
75%,25.421472,-99.117922,292.000000,119.000000
max,34.570000,0.000000,435.000000,150.000000


In [8]:

# Count distinct observations over requested axis.
estaciones.id.nunique(), estaciones._id.nunique(), estaciones.lat.nunique(), estaciones.long.nunique()

(185, 185, 180, 179)

In [9]:
# Estaciones son los datos obtenidos de la API, en formato Pandas DataFrame
# Hay estaciones con valores erróneos
estaciones.plot.scatter('long', 'lat', figsize=(8, 8))
mplleaflet.display()

In [20]:
# Show the measurement stations in a map

# Center the map
centro_lat, centro_lon = 22.396092, -101.731430

# Generate the map
folium_map = folium.Map(location=[centro_lat, centro_lon], zoom_start=5, tiles='cartodb positron')

# Populate the map 
for i, row in estaciones.iterrows():
    # generate the popup message that is shown on click (name, latitude and longitude)
    popup_text = f"<b> Nombre: </b> {row.nombre} <br> <b> Latitud: </b> {row.lat:.5f} <br> <b> Longitud: </b> {row.long:.5f} <br>"
    # Aggregate markers
    folium.CircleMarker(location=[row.lat, row.long], radius=5, tooltip=popup_text, fill=True, fill_opacity=0.4).add_to(folium_map)

# Save the map into a file
folium_map.save('data_html/estaciones_todas.html')

# Show/render the map
folium_map

In [11]:
# Guardamos los datos para utilizarlos más adelante
estaciones.to_csv('data_csv/data_estaciones_sinaica.csv', index=False)

## Obtener los registros de mediciones

Find out how many records there are, determine how many records we want to retrieve at a time and hence how many requests we need to make.

In [21]:
# Obtener las mediciones por hora de cada contaminante por estación (devuelve un json) y las ponemos en un numero de archivos
# 
# Aqui solo vamos a fijarnos en la estacion de la Merced, estacionesid = 256
#medidas_estacion_hora_request = client.makeCall('sinaica', {'pageSize':1000, 'parametro':'O3', 'estacionesid':256, 'page':1})
#https://api.datos.gob.mx/v2/sinaica?pageSize=1000&parametro=O3&estacionesid=256&page=1 
from datosgobmx import client
import pandas as pd

from pathlib import Path
import json

# issue a query, any really, just to find out how many records are there in total
# https://api.datos.gob.mx/v2/sinaica?pageSize=200&parametro=O3&estacionesid=256&page=1
medidas_estacion_hora_request = client.makeCall('sinaica', {'pageSize':200, 'parametro':'O3', 'estacionesid':256, 'page':1}) 

# this is the total number of records available. El 10 April 2020 habia 7573 registros
numero_medidas = int(medidas_estacion_hora_request['pagination']['total'])

# how many records we want to get in each request
medidas_por_pagina = 1500

# number of pages to retrieve = numero_medidas / medidas por pagina
import math         # because we need to round the number of pages to an integer, and up
numero_de_paginas = int(math.ceil(numero_medidas / medidas_por_pagina))
# print(numero_de_paginas)

Obtener los jsons

In [13]:
from tqdm import tqdm
from pathlib import Path

# el 10 April 2020 hay 7573 registros
for page in tqdm(range(1, numero_de_paginas+1)):
    filename = 'page_%i_sinaica_mediciones.json' %page
    filename = 'mediciones_sinaica_json/'+filename
    #print(filename)
    if not Path(filename).is_file():
        data_api = client.makeCall('sinaica', {'pageSize':1500, 'parametro':'O3', 'estacionesid':256, 'page':page})
        with open(filename, 'w') as outfile:
            json.dump(data_api, outfile)


100%|██████████| 6/6 [00:00<00:00, 1670.04it/s]


Formatear json a csv

In [2]:
import json
from glob import glob
from joblib import Parallel, delayed

def parse_mediciones_json(json_file):
    "Función auxiliar para convertir los jsons de mediciones de calidad de aire en Pandas Data Frame"
    # Abrimos el archivo json descargado previamente
    with open(json_file, 'r') as aux:
        results = json.load(aux)['results'] # Los resultados vienen en la seccion *results*
    
    # Vamos a guardar los datos de resultados en una lista para después concatenarlos en un DataFrame
    pre_data = []
    
    for r in results:
        # Obtenemos DataFrame previo y lo agregamos
        aux = pd.DataFrame.from_dict(r, orient='index').T
        pre_data.append(aux)
    
    # Concatenamos todos los datos y devolvemos el DataFrame, siempre que haya datos
    if len(pre_data)>0:
        pre_data = pd.concat(pre_data, ignore_index=True)
        return pre_data
    
    # Si por alguna razón no hay datos imprimimos el resultado y no devolvemos nada
    else:
        print('NO DATA FOR ', json_file)
        return

Vamos a dar formato a los archivos en dos conjuntos para evitar problemas de memoria. El primer conjunto va a tener los datos de los primeros 500 archivos json y el segundo, de los restantes. Para realizar esta tarea de forma más veloz, utilizamos una librería que permite realizar tareas en paralelo llamada joblib (ver https://joblib.readthedocs.io/en/latest/parallel.html). Luego, se guarda todo en un archivo maestro.

In [15]:
name = 'mediciones_sinaica_json/page_%i_sinaica_mediciones.json'

c = 1

for i, j in zip([1, 4], [4, 6]):
    print('Chunk', c)
    aux_list = [name %n for n in range(i, j)]
    print(aux_list)

    sinaica_mediciones = Parallel(n_jobs=-1, verbose=8)(delayed (parse_mediciones_json)(f) for f in aux_list)

    sinaica_mediciones = pd.concat(sinaica_mediciones, ignore_index=True)
    print(sinaica_mediciones.shape)
    sinaica_mediciones.to_csv('mediciones_sinaica_csv/sinaica_mediciones_%i_%i.csv' %(i, j),
                              index=False)
    c+=1

Chunk 1
['mediciones_sinaica_json/page_1_sinaica_mediciones.json', 'mediciones_sinaica_json/page_2_sinaica_mediciones.json', 'mediciones_sinaica_json/page_3_sinaica_mediciones.json']
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    7.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    7.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
(4500, 11)
Chunk 2
['mediciones_sinaica_json/page_4_sinaica_mediciones.json', 'mediciones_sinaica_json/page_5_sinaica_mediciones.json']
(3000, 11)
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    5.5s finished


In [ ]:
# alternative 
"""
# ruta donde guardamos los jsons, vamos a formatearlo mas adelante para leer cada archivo
name = 'mediciones_sinaica_json/page_%i_sinaica_mediciones.json'

aux_list = [name %i for i in range(1, 17)]
#print(aux_list)

frames = [ parse_mediciones_json(json_file) for json_file in aux_list]


# Concatenamos todo
sinaica_mediciones = pd.concat(frames, ignore_index=True)
# print(sinaica_mediciones.shape)   # (7573, 11)


# Guardamos el archivo
sinaica_mediciones.to_csv('mediciones_sinaica_csv/mediciones_sinaica.csv', index=False)


from glob import glob

all_data_mediciones = []

for file in glob('mediciones_sinaica_csv/*.csv'):
    #aux = pd.read_csv(file, usecols=['estacionesid', 'fecha', 'hora', 'city', 'state', 'parametro', 'valororig', 'validoorig'])
    aux = pd.read_csv(file)
    all_data_mediciones.append(aux)

all_data_mediciones = pd.concat(all_data_mediciones)
#all_data_mediciones.info(verbose=True)
"""


In [7]:
all_data_mediciones = []
for file in glob('mediciones_sinaica_csv/*.csv'):
    aux = pd.read_csv(file,
                      usecols = ['estacionesid', 'fecha', 'hora', 'city', 'state',
                                 'parametro', 'valororig', 'validoorig'])
    all_data_mediciones.append(aux)
    
all_data_mediciones = pd.concat(all_data_mediciones)
print(all_data_mediciones.shape)
all_data_mediciones.head()

(10573, 8)


,validoorig,city,state,valororig,parametro,estacionesid,hora,fecha
0,1,Valle de México,Ciudad de México,0.007,O3,256,9,2018-08-10
1,1,Valle de México,Ciudad de México,0.016,O3,256,10,2018-08-10
2,1,Valle de México,Ciudad de México,0.024,O3,256,11,2018-08-10
3,1,Valle de México,Ciudad de México,0.039,O3,256,12,2018-08-10
4,1,Valle de México,Ciudad de México,0.060,O3,256,13,2018-08-10


In [8]:
all_data_mediciones.estacionesid.nunique()

1

In [9]:
# all_data_mediciones.to_csv('/data_mediciones_todas_estaciones.csv', index=False)
# Guardamos los datos
all_data_mediciones.to_csv('data_csv/merced_o3_master.csv', index=False)